# Création des bdd comportant la variable conso_moyenne

Dans ce notebook, nous allons créer les tables contenant la variable consommation moyenne suivante : $conso\_moyenne=\frac{conso\_totale}{nb\_point\_soutirage}$

In [1]:
import pandas as pd

Le problème est que le nombre de points de soutirage par région pour une plage de puissance donnée varie légèrement toutes les 30min. Contrairement à ce que dit enedis, il n'est donc pas parfaitement égale. Nous allons alors attribuer un "indice" de points de soutirage pour chaque donnée de consommation afin de savoir à quel regroupement de point de soutirage il correspondrait. Ensuite, nous effectuerons une moyenne dans chaque groupement pour obtenir le nombre de point de soutirage moyen par groupement.


**Exemple :**

- 20/06/2020 00:00 Bretagne 648983 points de soutirage
- 20/06/2020 08:00 Bretagne 648933 points de soutirage

On remarque que les points diffèrent légèrement. Nous ne pourrons pas regrouper ces deux données en se basant uniquement sur le nombre de points de soutirage. Il faut donc leur attribuer un numéro de groupement, ce qu'on fait dans ce notebook. 

## Import des données :

In [2]:
enedis_final=pd.read_csv("Données/enedis_filtered.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'Données/enedis_filtered.csv'

On veut créer la variable consommation moyenne (par nombre de points de soutirage).    
On va attribuer un indice à chaque région afin de trouver la moyenne du nombre de points de soutirage par région :

In [5]:
enedis_final["indice_groupe"]=0

### Table de température (sert plus tard)

In [6]:
temperature=enedis_final[["date","region_x","tmoy"]]

### Construction de la liste de référence du nombre de points de soutirage

On commence par travailler sur chaque région pour un jour donné. Pour une région, il peut y avoir plusieurs nombre de point de soutirage différent. En effet, ce dernier peut varier légèrement d'une demie-heure à l'autre, ce qui poser problème dans la création de notre nouvelle variable. Donc on va compter l'apparition de ces différents points de soutirage et on fera ensuite une moyenne.

In [7]:
france_point=enedis_final[(enedis_final.date=="2020-06-20")].groupby(["region_x","nb_points_soutirage"])["horodate"].count().reset_index(name="count_apparition")
france_point["id_soutirage"]=1
france_point.sort_values("nb_points_soutirage",inplace=True)

In [8]:
france_point

,region_x,nb_points_soutirage,count_apparition,id_soutirage
6,Centre-Val de Loire,409567,2,1
7,Centre-Val de Loire,409619,20,1
2,Bourgogne-Franche-Comté,550060,3,1
3,Bourgogne-Franche-Comté,550133,24,1
13,Normandie,602675,1,1
14,Normandie,602736,21,1
4,Bretagne,648933,2,1
5,Bretagne,648983,14,1
19,Pays de la Loire,703823,3,1
20,Pays de la Loire,703886,19,1


In [12]:
def plage(value,epsilon,ref):
    ''' 
    Fonction qui permet de savoir si la valeur est bien dans la plage [ref - epsilon ; ref + epsilon]  '''
    
    if (ref-epsilon<=value<=ref+epsilon):
        return True
     

Ensuite on parcourt notre base afin de trouver quels sont les réels valeurs des points de soutirage.

In [13]:
point_soutirage_ref=france_point.iloc[0][1]
region=france_point.iloc[0][0]
groupe_indice=1
list_ref=[]
for i in range(1,len(france_point)):
    if plage(france_point["nb_points_soutirage"][i],300,point_soutirage_ref):
        france_point["id_soutirage"][i]=groupe_indice
    else:
        groupe_indice+=1
        france_point["id_soutirage"][i]=groupe_indice
        list_ref.append([region,point_soutirage_ref])
    point_soutirage_ref=france_point["nb_points_soutirage"][i]
    region=france_point["region_x"][i]
    

C:\Users\elelo\AppData\Local\Temp/ipykernel_21868/970881880.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  france_point["id_soutirage"][i]=groupe_indice
C:\Users\elelo\AppData\Local\Temp/ipykernel_21868/970881880.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  france_point["id_soutirage"][i]=groupe_indice


Voici la liste des valeurs des points de soutirage. Pour chaque région, si d'autres valeurs sont plus ou moins proches de celles comprises dans cette liste, alors elles se verront attribuer le même groupe d'indice.

In [16]:
list_ref

[['Centre-Val de Loire', 409567],
 ['Auvergne-Rhône-Alpes', 1686608],
 ['Bourgogne-Franche-Comté', 550133],
 ['Bretagne', 648983],
 ['Centre-Val de Loire', 409619],
 ['Grand-Est', 816010],
 ['Hauts-de-France', 1101626],
 ['Normandie', 602736],
 ['Nouvelle Aquitaine', 996761],
 ['Occitanie', 1142850],
 ['Pays de la Loire', 703886],
 ["Provence-Alpes-Côte d'Azur", 997436],
 ['Île-de-France', 3099647]]

Fonction pour attribuer l'indice du point de soutirage à chaque ligne de notre bdd en se basant sur la liste de référence comportant tous les points de soutirage :

In [10]:
def attribution(value,region,epsilon,list_ref):
    for i in range(len(list_ref)):
        if (list_ref[i][0]==region) & (list_ref[i][1]-epsilon<=value<=list_ref[i][1]+epsilon):
            return str(i)

In [11]:
enedis_final

,horodate,region_x,code_region,total_energie_soutiree_wh,plage_de_puissance_souscrite,nb_points_soutirage,jour_max_du_mois_0_1,semaine_max_du_mois_0_1,profil,profil_type,...,Médiane du revenu disponible par UC 2018,Population au dernier recensement 2018,Part des 15-29 ans 2018,Part des 30-44 ans 2018,Part des 45-59 ans 2018,Part des 60-74 ans 2018,Part des 75 ans et plus 2018,Taux de chômage des 15 ans et plus (RP) 2018,tmoy,indice_groupe
0,2020-06-04T00:00:00+00:00,Auvergne-Rhône-Alpes,84,130666817.0,P2: ]3-6] kVA,1685439,0,0,RES1 (+ RES1WE),RES,...,22480.0,7994459.0,17.6,18.8,19.8,16.2,9.4,11.4,16.48,0
1,2020-06-04T00:00:00+00:00,Bourgogne-Franche-Comté,27,44987629.0,P2: ]3-6] kVA,550076,0,0,RES1 (+ RES1WE),RES,...,21480.0,2807807.0,16.2,17.3,20.1,18.6,10.9,12.0,17.17,0
2,2020-06-04T00:00:00+00:00,Hauts-de-France,32,92853270.0,P2: ]3-6] kVA,1101239,0,0,RES1 (+ RES1WE),RES,...,20110.0,6004108.0,18.5,18.8,19.5,15.9,7.9,16.3,13.66,0
3,2020-06-04T00:00:00+00:00,Île-de-France,11,238127892.0,P2: ]3-6] kVA,3097303,0,0,RES1 (+ RES1WE),RES,...,23860.0,12213447.0,20.0,21.2,19.4,13.2,6.8,12.1,14.51,0
4,2020-06-04T00:00:00+00:00,Grand-Est,44,69687114.0,P2: ]3-6] kVA,815710,0,0,RES1 (+ RES1WE),RES,...,21610.0,5550389.0,17.5,18.3,20.6,17.1,9.3,13.3,15.82,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121271,2021-07-30T23:30:00+00:00,Hauts-de-France,32,97406596.0,P2: ]3-6] kVA,1119748,0,0,RES1 (+ RES1WE),RES,...,20110.0,6004108.0,18.5,18.8,19.5,15.9,7.9,16.3,16.72,0
121272,2021-07-30T23:30:00+00:00,Île-de-France,11,225060904.0,P2: ]3-6] kVA,3133979,0,0,RES1 (+ RES1WE),RES,...,23860.0,12213447.0,20.0,21.2,19.4,13.2,6.8,12.1,18.68,0
121273,2021-07-30T23:30:00+00:00,Bourgogne-Franche-Comté,27,48132464.0,P2: ]3-6] kVA,555237,0,0,RES1 (+ RES1WE),RES,...,21480.0,2807807.0,16.2,17.3,20.1,18.6,10.9,12.0,19.16,0
121274,2021-07-30T23:30:00+00:00,Provence-Alpes-Côte d'Azur,93,91928076.0,P2: ]3-6] kVA,1026967,0,0,RES1 (+ RES1WE),RES,...,21350.0,5052832.0,16.1,17.9,20.1,18.0,11.1,14.0,24.61,0


In [12]:
enedis_final["indice_groupe"]=enedis_final.apply(lambda x: plage(x.nb_points_soutirage,x.region_x,50000,list_ref),axis=1)

### Construction de la table contenant les totaux des points de soutirage : 

Moyenne du nombre de points de soutirage par sous groupes :

In [13]:
table_region_points=enedis_final.groupby("indice_groupe")["nb_points_soutirage"].mean().reset_index()

## Consommation moyenne pour la france :

In [14]:
nb_pts=table_region_points.nb_points_soutirage.sum()

Création de la variable consommation moyenne pour la france (conso/nb de point soutirage)

In [16]:
conso_totale=enedis_final.groupby("date")["total_energie_soutiree_wh"].sum().reset_index()
conso_totale["conso_totale_moyenne"]=conso_totale.total_energie_soutiree_wh/nb_pts


On merge temperature :

In [23]:
temperature_moy=temperature.groupby("date")["tmoy"].mean().reset_index()

In [18]:
conso_france=conso_totale.merge(temperature_moy,on="date",how="right")

In [19]:
conso_france.to_csv("Données/data_conso_moyenne/data_conso_france.csv")

## Consommation moyenne pour les régions :

On crée maintenant les tables de conso moyenne pour chaque région :

In [82]:
def creation_table_conso_moyenne(region,nom_court):
    
    enedis_region=enedis_final[enedis_final.region_x==region]

    indice_groupement=enedis_region.indice_groupe.values[0] 
    nb_total_points_soutirage_region=table_region_points[table_region_points.indice_groupe==indice_groupement]["nb_points_soutirage"].values[0]

    conso_totale_region=enedis_region.groupby("date")["total_energie_soutiree_wh"].sum().reset_index()
    conso_totale_region["conso_totale_moyenne"]=conso_totale_region.total_energie_soutiree_wh/nb_total_points_soutirage_region


    temperature_region=temperature[temperature.region_x==region]
    temperature_region.drop_duplicates(inplace=True)

    conso_region=conso_totale_region.merge(temperature_region,on="date",how="right")

    conso_region.to_csv("Données/data_conso_moyenne/data_conso_"+nom_court+".csv",index=False)

In [83]:
creation_table_conso_moyenne("Île-de-France","IDF")
creation_table_conso_moyenne("Hauts-de-France","HDF")
creation_table_conso_moyenne("Provence-Alpes-Côte d'Azur","PACA")
creation_table_conso_moyenne("Centre-Val de Loire","centre_val_de_loire")
creation_table_conso_moyenne("Normandie","Normandie")
creation_table_conso_moyenne("Occitanie","Occitanie")
creation_table_conso_moyenne("Auvergne-Rhône-Alpes","Auvergne-Rhône-Alpes")
creation_table_conso_moyenne("Bourgogne-Franche-Comté","Bourgogne_franche_comte")
creation_table_conso_moyenne("Nouvelle Aquitaine","Nouvelle_Aquitaine")
creation_table_conso_moyenne("Bretagne","Bretagne")
creation_table_conso_moyenne("Grand-Est","grand_est")
creation_table_conso_moyenne("Pays de la Loire","Pays_Loire")

c:\Users\elelo\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
